## Chapter 8

## Optimization for Training Deep Models

### 8.1 How Learning Differes from Pure Optimization

<strong>ML/DL Optimization </strong>: <br>
 1. acts indirectly - reduce cost function $J(\theta)$ to improve $P$

<strong>Pure Optimization</strong>: <br>
1. acts directly - minimize $J$ is the goal

Objective function: <br>


\begin{equation*}
J^{*}(\theta) =  \mathop{\mathbb{E}}_{(x,y) \sim p_{data}} L(f(x;\theta),y)
\end{equation*}


#### 8.1.1 Empirical Risk Minimization

Above equation is known as <strong> risk </strong>
<br> but in reality, the true underlying distribution $p_{data}$ is not known
<br> replacing the true distribution $p(x,y)$ with empirical distribution $\hat{p}(x,y)$

<strong> empirical risk: </strong>
\begin{equation*}
\mathop{\mathbb{E}}_{x,y \sim \hat{p}_{data}(x,y)} [L(f(x;\theta),y)] = \frac{1}{m} \sum\limits_{i=1}^m L(f(x^{(i)};\theta), y^{(i)})
\end{equation*}
where $m$ is the number of training examples

However, empirical risk minimization is prone to overfitting <br>
 - models with High Capacity can simply memorize the training set
 - many useful loss functions such as 0-1 loss have no useful derivatives
<br>$\therefore$ emprical risk minimization is rarely used in DL

#### 8.1.2 Surrogate Loss Functions and Early Stopping

<strong> surrogate loss function </strong> is used as a proxy, and has advantages over the original loss function. 
<br> Negative Log-Likelihood is typically used as a surrogate for the 0-1 loss.

Important Difference between Optimization in General and Optimization in ML/DL:
<br> ML usually minimizes a surrogate loss function but <strong> halts when a convergence criterion based on early stopping is satisfied </strong> 
<br>whereas for the pure optimization, it is considered to have converged when the <strong> gradient becomes very small </strong>

#### 8.1.3 Batch and Minibatch Algorithms

ML: Objective function usually decomposes as a <strong> sum over the training examples </strong>

For Example, MLE: <br>

\begin{equation*}
\theta_{ML} = arg\max\limits_{\theta} \sum\limits_{i=1}^m log p_{model} (x^{(i)}, y^{(i)}; \theta) 
\end{equation*}


Maximizing this sum is equivalent to maximizing the Expectation over the Empirical distribution defined by the training set: <br>
\begin{equation*}
\mathop{\mathbb{E}}_{x,y \sim \hat{p}_{data}} log p_{model}(x, y; \theta)
\end{equation*}

Most commonly used property is the gradient: <br>
\begin{equation*}
\nabla_{\theta} \:J(\theta) = \mathop{\mathbb{E}}_{x,y \sim \hat{p}_{data}} \nabla_{\theta}\: log\: p_{model} (x, y; \theta)
\end{equation*}

Computing this expectation exactly is very expensive <br>
 - instead, compute these expectations by randomly sampling a small number of examples from the dataset, then taking the Average over only those examples

1. The Standard Error of the Mean is given by: <br>
\begin{equation*}
SE(\hat{\mu}_m) = \sqrt{Var\bigg[\frac{1}{m}\sum\limits_{i=1}^mx^{(i)}\bigg]} = \frac{\sigma}{\sqrt{m}}
\end{equation*}
<br> where $\sigma$ is the true standard deviation of the value of the samples

The denominator of $\sqrt{m}$ shows that there iare <strong> less than linear returns </strong> to using more examples to estimate the gradient.

2. We may find large number of examples that all make very similar contributions to the gradient - <strong>redundant samples</strong>

<strong> batch / deterministic </strong>: use entire training set <br>
<strong> online </strong>: use examples drawn from a continuous stream of new data <br>
<strong> minibatch / stochastic </strong>: use more than one but less than all of training examples <br>

minibatch sizes are decided by following factors:
- larger batches = more accurate estimate of gradient, but with less than linear returns
- small batches do not utilize multi-core architectures
- small batches can offer a regularizing effect - due to noise they add to learning process. Generalization error is often best for a batch size of 1. But learning will take more time - more steps + reduced learning rate to maintain stability due to high variance in gradient estimate

Update methods based only on the gradient $\mathbf{g}$ are usually relatively robust and can handle smaller batch sizes like 100. <br>
Second-order methods that compute updates such as $\mathbf{H^{-1}g}$ typically require much larger batch sizes like 10,000. - to minimize fluctuations in the estimates of *$\mathbf{H^{-1}g}$*

If $\mathbf H$ is poorly conditioned, very small changes in the estimate of $\mathbf g$ can cause large changes in the update $\mathbf H^{-1}g$, even if $\mathbf H$ were estimated perfectly 

Recall from **4.2 Poor Conditioning**: <br>
$f(x) \; = \; A^{-1}x $ <br>
$A \; \in \; \mathbb{R}^{n\times n} $ has an eigenvalue decomposition<br>
condition number is: $\max \limits_{i,j}\|\frac{\lambda_i}{\lambda_j}\|$
<br> This is the ratio of the magnitude of the largest and smallest eigenvalue
<br> when condition number is large, matrix inversion is particularly sensitive to error in the input
<br> Poorly conditioned matrices amplify pre-existing erros when we multiply by the true matrix inverse.

Important to select minibatches **randomly**.

### 8.2 Challenges in Neural Network Optimization

#### 8.2.1 Ill-Conditioning

Most prominent challenge is ill-conditioning of the Hessian matrix $H$ <br>
It can manifest by causing SGD to get 'stuck' in the sense that even very small steps increase the cost function

Newton's method is an excellent tool for minimizing convex functions with poorly conditioned Hessian matrices, but in the subsequent sections we will argue that Newton’s method requires significant modification before it can be applied to neural networks.

#### 8.2.2 Local Minima

Nearly any deep model is essentially guaranteed to have an extremely large number of local minima. However, as we will see, this is not necessarily a major problem.

Neural networks and any models with multiple equivalently parametrized latent
variables all have multiple local minima because of the **model identifiability**
problem. A model is said to be identifiable if a sufficiently large training set can rule out all but one setting of the model’s parameters. Models with latent variables are often not identifiable because we can obtain equivalent models by exchanging latent variables with each other.

If we have m layers with n units each, then there are n ! m ways of arranging the hidden units. This kind of non-identifiability is known as **weight space symmetry.**

These model identifiability issues mean that there can be an extremely large
or even uncountably infinite amount of local minima in a neural network cost
function. However, all of these local minima arising from non-identifiability are
equivalent to each other in cost function value. As a result, these local minima are not a problematic form of non-convexity.

For sufficiently large neural networks, most local minima have a
low cost function value, and that it is not important to find a true global minimum rather than to find a point in parameter space that has low but not minimal cost

### 8.2.3 Plateaus, Saddle Points and Other Flat Regions

For many high-dimensional non-convex functions, local minima (and maxima)
are in fact rare compared to another kind of point with zero gradient: a **saddle
point.**

At a saddle point, the Hessian matrix has both positive and negative eigenvalues. Points lying along eigenvectors associated withpositive eigenvalues have greater cost than the saddle point, while points lying along negative eigenvalues have lower value. 

In low- dimensional spaces, local minima are common. In higher dimensional spaces, local minima are rare and saddle points are more common. For a function $f : \mathbb{R}^n → \mathbb{R}$ of this type, the expected ratio of the number of saddle points to local minima grows exponentially with n .

To understand the intuition behind this behavior, observe that the Hessian matrix at a local minimum has only positive eigenvalues. The Hessian matrix at a saddle point has a mixture of positive and negative eigenvalues. Imagine that the sign of each eigenvalue is generated by flipping a coin. In a single dimension, it is easy to obtain a local minimum by tossing a coin and getting heads once. In n -dimensional space, it is exponentially unlikely that all n coin tosses willbe heads

Gradient descent is designed to move “downhill” and is not explicitly designed
to seek a critical point. Newton’s method, however, is designed to solve for a
point where the gradient is zero. Without appropriate modification, it can jump
to a saddle point. The proliferation of saddle points in high dimensional spaces
presumably explains why second-order methods have not succeeded in replacing
gradient descent for neural network training. 

Dauphin et al. 2014 introduced a **saddle-free Newton method** for second-order optimization and showed that it improves significantly over the traditional version. Second-order methods remain difficult to scale to large neural networks, but this saddle-free approach holds promise if it could be scaled.

#### 8.2.4 Cliffs and Exploding Gradients

Neural networks with many layers often have extremely steep regions resembling
cliffs. These result from the multiplication of several large weights together.

The cliff can be dangerous whether we approach it from above or from below,
but fortunately its most serious consequences can be avoided using the gradient
clipping heuristic

Cliff
structures are most common in the cost functions for recurrent neural networks,
because such models involve a multiplication of many factors, with one factor
for each time step. 

#### 8.2.5 Long-Term Dependencies

Suppose that $W$ has an eigendecomposition $W\;=\;Vdiag(\lambda)V^{-1}$

$W^{t}\;=\;(Vdiag(\lambda)V^{-1})^{t}\;=\;Vdiag(\lambda)^{t}V^{-1}$

**Vanishing and exploding gradient problem** refers to the fact that gradients
through such a graph are also scaled according to diag ( λ ) t . Vanishing gradients
make it difficult to know which direction the parameters should move to improve
the cost function, while exploding gradients can make learning unstable. The cliff
structures described earlier that motivate gradient clipping are an example of the
exploding gradient phenomenon.

Tt is not surprising that $x^{T}W^{t}$ will eventually discard all components of **$x$** that are orthogonal to the principal eigenvector of $W$

#### 8.2.6 Inexact Gradients

Most optimization algorithms are designed with the assumption that we have
access to the exact gradient or Hessian matrix. In practice, we usually only have
a noisy or even biased estimate of these quantities. Nearly every deep learning
algorithm relies on sampling-based estimates at least insofar as using a minibatch
of training examples to compute the gradient.

#### 8.2.7 Poor Correspondence between Local and Global Structure

#### 8.2.8 Theoretical Limits of Optimization

Several theoretical results show that there are limits on the performance of any
optimization algorithm we might design for neural networks (Blum and Rivest,
1992; Judd 1989; Wolpert and MacReady 1997). Typically these results have
little bearing on the use of neural networks in practice.

### 8.3  Basic Algorithms

#### 8.3.1 Stochastic Gradient Descent

**while** stopping criterion not met **do**
> Sample minibatch 
    <br>Compute gradient estimate: $\hat{g} \leftarrow +\frac{1}{m}\nabla_\theta \Sigma_{i} L(f(x^{(i)};\theta), y^{(i)}) $
    <br>Apply update: $\theta \leftarrow \theta - \epsilon\hat{g}$  

**end while**

 SGD gradient estimator introduces a source of noise (the
random sampling of m training examples) that does not vanish even when we arrive
at a minimum. 

A sufficient condition to guarantee convergence of SGD is that

$ \sum\limits_{k=1}^{\infty} \epsilon_k = \infty$ and,

$ \sum\limits_{k=1}^{\infty} \epsilon_{k}^{2} < \infty$

In practice, it is common to decay the learning rate linearly until iteration $\tau$

$\epsilon_k\;=\;(1-\alpha)\epsilon_0 + \alpha\epsilon_\tau$

with $\alpha\;=\;\frac{k}{\tau}$

#### 8.3.2 Momentum

**while** stopping criterion not met **do**
> Sample minibatch 
    <br>Compute gradient estimate: $ \hat{g} \leftarrow +\frac{1}{m}\nabla_\theta \Sigma_{i} L(f(x^{(i)};\theta), y^{(i)}) $
    <br> Compute velocity update: $v \leftarrow \alpha v - \epsilon g$
    <br>Apply update: $\theta \leftarrow \theta + v$  

**end while**

#### 8.3.3 Nesterov Momentum

**while** stopping criterion not met **do**
> Sample minibatch 
    <br> Apply interim update: $\tilde{\theta} \leftarrow \theta + \alpha v$
    <br>Compute gradient estimate (at interim point): $ g \leftarrow +\frac{1}{m}\nabla_\theta \Sigma_{i} L(f(x^{(i)};\theta), y^{(i)}) $
    <br> Compute velocity update: $v \leftarrow \alpha v - \epsilon g$
    <br>Apply update: $\theta \leftarrow \theta + v$  

**end while**

### 8.4 Parameter Initialization Strategies

Glorot and Bengio
(2010) suggest using the **normalized initialization**

$W_{i,j} \sim U \bigg(-\sqrt{\frac{6}{m+n}},\sqrt{\frac{6}{m+n}}\bigg)$

### 8.5 Algorithms with Adaptive Learning Rates

#### 8.5.1 AdaGrad

**while** stopping criterion not met **do**
> Sample minibatch 
    <br>Compute gradient: $ g \leftarrow \frac{1}{m}\nabla_\theta \Sigma_{i} L(f(x^{(i)};\theta), y^{(i)}) $
    <br>Accumulate squared gradient: $r \leftarrow r + g \odot g $
    <br> Compute update: $\varDelta\theta \leftarrow - \frac{\epsilon}{\delta + \sqrt{r}} \odot g$
    <br>Apply update: $\theta \leftarrow \theta + \varDelta\theta$  

**end while**

#### 8.5.2 RMSProp

**while** stopping criterion not met **do**
> Sample minibatch 
    <br>Compute gradient: $ g \leftarrow \frac{1}{m}\nabla_\theta \Sigma_{i} L(f(x^{(i)};\theta), y^{(i)}) $
    <br>Accumulate squared gradient: $r \leftarrow \rho r + (1-\rho)g \odot g $
    <br> Compute update: $\varDelta\theta \leftarrow - \frac{\epsilon}{\sqrt{\delta+r}} \odot g$ 
    <br>Apply update: $\theta \leftarrow \theta + \varDelta\theta$  

**end while**

#### 8.5.3 Adam

**while** stopping criterion not met **do**
> Sample minibatch 
    <br>Compute gradient: $ g \leftarrow \frac{1}{m}\nabla_\theta \Sigma_{i} L(f(x^{(i)};\theta), y^{(i)}) $
    <br>$t \leftarrow t + 1 $
    <br>Update biased first moment estimate: $s \leftarrow \rho_1 s + (1-\rho_1)g$
    <br>Update biased second moment estimate: $r \leftarrow \rho_2 r + (1-\rho_2)g\odot g$
    <br> Correct bias in first moment: $\hat{s} \leftarrow \frac{s}{1-\rho_1^t}$
    <br> Correct bias in second moment: $\hat{r} \leftarrow \frac{r}{1-\rho_2^t}$
    <br> Compute update: $\varDelta\theta \leftarrow - \epsilon \frac{\hat{s}}{\sqrt{\hat{r}} + \delta}$ 
    <br>Apply update: $\theta \leftarrow \theta + \varDelta\theta$  

**end while**

### 8.6 Approximate Second-Order Methods

Empirical Risk:<br>
$J(\theta) = \mathbb{E}_{x,y\sim\hat{p}_data (x,y)} [L(f(x;\theta),y)] = \frac{1}{m}\sum\limits_{i=1}^{m}L(f(x^{(i)};\theta),y^{(i)})
$

#### 8.6.1 Newton's Method

In contrast to first-order methods, second-order methods make use of second derivatives to improve optimization.

Newton’s method is an optimization scheme based on using a second-order Tay-
lor series expansion to approximate $J(\theta)$ near some point $\theta_0$ , ignoring derivatives of higher order:

$
J(\theta) \approx J(\theta_0) + (\theta - \theta_0)^{T}\nabla_\theta J(\theta_0) + \frac{1}{2}(\theta - \theta_0)^{T} H (\theta - \theta_0)
$

Solve for the critical point of this function, we obtain the **Newton parameter update rule:**

$
    \theta^{*} = \theta_0 - H^{-1}\nabla_\theta J(\theta_0)
$

Thus for a locally quadratic function (with positive definite $H$), by rescaling
the gradient by $H^{−1}$ , Newton’s method jumps directly to the minimum.

**while** stopping criterion not met **do**
> Compute gradient: $ g \leftarrow \frac{1}{m}\nabla_\theta \Sigma_{i} L(f(x^{(i)};\theta), y^{(i)}) $
    <br>Compute Hessian: $ H \leftarrow \frac{1}{m}\nabla_\theta^2 \Sigma_{i} L(f(x^{(i)};\theta), y^{(i)}) $
    <br>Compute Hessian Inverse: $H^{-1}$
    <br>Compute update: $\varDelta\theta = -H^{-1}g$
    <br>Apply update: $\theta = \theta + \varDelta\theta$  

**end while**

Common regularization strategies include adding a constant, $\alpha$ along the diagonal of the Hessian. The regularized update becomes:

$
    \theta^* = \theta_0 - [H(f(\theta_0)) + \alpha I]^{-1} \nabla_\theta f(\theta_0)
$

 application of Newton’s method for training large neural
networks is limited by the significant computational burden it imposes

The number of elements in the Hessian is squared in the number of parameters, so with k parameters, Newton’s method would require the inversion of a k k ×
matrix—with computational complexity of $O(k^3)$. 

Also, since the parameters will change with every update, the inverse Hessian has to be computed at *every training iteration*

#### 8.6.2 Conjugate Gradients

Conjugate gradients is a method to efficiently avoid the calculation of the inverse Hessian by iteratively descending conjugate directions.

 we seek to find a search direction that is conjugate to the previous line search direction, i.e. it will not undo progress made in that direction.

 By definition, at
the minimum of the objective along a given direction, the gradient at the final point is
orthogonal to that direction.

at training iteration $t$, the next search direction $d_t$ takes the form:
<br>
$d_t = \nabla_\theta J(\theta) + \beta_t d_{t-1}$
<br>where $\beta_t$ is a coefficient whose magnitude controls how much of the direction, $d_{t-1}$ we should add back to the current search direction.

Two directions, $d_t$ and $d_{t−1}$ , are defined as conjugate if $d_t^T H d_{t-1} = 0$, where $H$ is the Hessian matrix.

The straightforward way to impose conjugacy would involve calculation of the
eigenvectors of $H$ to choose $\beta_t$, which would not satisfy our goal of developing a method that is more computationally viable than Newton’s method for large problems.

Two popular methods for computing the $\beta_t$ are:
<br>
1. Fletcher-Reeves:<br>
 $\beta_t = \ \cfrac{\nabla_\theta J(\theta_t)^T \nabla_\theta J(\theta_t)}
 {\nabla_\theta J(\theta_{t-1})^T \nabla_\theta J(\theta_{t-1}}$
<br> <br>
2. Polak-Ribeiere:<br>
 $\beta_t = \ \cfrac{(\nabla_\theta J(\theta_t) - \nabla_\theta J(\theta_{t-1}))^T \nabla_\theta J(\theta_t)}
 {\nabla_\theta J(\theta_{t-1})^T \nabla_\theta J(\theta_{t-1}}$

**The conjugate gradient method**

**while** stopping criterion not met **do**

>Initialize the gradient $g_t = 0$ 
    <br>Compute gradient: $ g_t \leftarrow \frac{1}{m}\nabla_\theta \Sigma_{i} L(f(x^{(i)};\theta), y^{(i)}) $
    <br>Compute $\beta_t = \cfrac{(g_t - g_{t-1})^T g_t}{g_{t-1}^T g_{t-1}} $
    <br>Compute search direction: $ \rho_t = -g_t + \beta_t \rho_{t-1} $
    <br> Perform line search to find: $ \epsilon^* = argmin_\epsilon \frac{1}{m} \sum\limits_{i=1}^{m}L(f(x^{(i)};\theta_t + \epsilon \rho_t), y^{(i)}) $
    <br>Apply update: $\theta_{t+1} = \theta_t + \epsilon^* \rho_t$
    <br>$t \leftarrow t + 1 $

**end while**

**Nonlinear Conjugate Gradients:** <br>
Without any assurance that the objective is quadratic, the conjugate directions are no longer assured to remain at the minimum of the objective for previous
directions. As a result, the nonlinear conjugate gradients algorithm includes
occasional resets where the method of conjugate gradients is restarted with line
search along the unaltered gradient.

#### 8.6.3 BFGS

**Broyden-Fletcher-Goldfarb-Shanno**  algorithm attempts to
bring some of the advantages of Newton’s method without the computational
burden. In that respect, BFGS is similar to the conjugate gradient method.
However, BFGS takes a more direct approach to the approximation of Newton’s
update

The approach adopted by quasi-Newton methods (of which
the BFGS algorithm is the most prominent) is to approximate the inverse with
a matrix $M_t$ that is iteratively refined by low rank updates to become a better
approximation of $H^{-1}$ .

Once the inverse Hessian approximation $M_t$ is updated, the direction of descent $\rho_t$ is determined by $\rho_t = M_t g_t$.  A line search is performed in this direction to determine the size of the step, $\epsilon^*$ , taken in this direction. The final update to the parameters is given by: <br>
$\theta_{t+1} = \theta_t + \epsilon^* \rho_t$

Unlike conjugate gradients, the success of the approach is not heavily dependent
on the line search finding a point very close to the true minimum along the line.
Thus, relative to conjugate gradients, BFGS has the **advantage** that it can spend **less time refining each line search**. On the other hand, the BFGS algorithm must **store the inverse Hessian matrix, $M$ , that requires $O(n^2)$ memory,** making BFGS
impractical for most modern deep learning models that typically have millions of
parameters.

 The L-BFGS algorithm computes the approximation $M$ using the same method as the BFGS algorithm, but beginning with the assumption that $M^{(t−1)}$ is the identity matrix, rather than storing the approximation from one step to the next.

The L-BFGS strategy with no storage described here can be
generalized to include more information about the Hessian by storing some of the
vectors used to update $M$ at each time step, which costs only $O(n)$per step.

### 8.7 Optimization Strategies and Meta-Algorithms

#### 8.7.1 Batch Normalization

Batch normalization (Ioffe and Szegedy 2015) is one of the most exciting recent 
innovations in optimizing deep neural networks and it is actually **not an optimization algorithm at all**. Instead, it is a method of **adaptive reparametrization**, motivated by the difficulty of training very deep models.

Very deep models involve the composition of several functions or layers. The
gradient tells how to update each parameter, under the assumption that the other
layers do not change. In practice, we update all of the layers simultaneously.

 This makes it very hard to choose an appropriate learning rate, because the effects of an update to the parameters for one layer depends so strongly on all of the other layers

Batch normalization provides an elegant way of reparametrizing almost any deep
network. The reparametrization significantly reduces the problem of coordinating
updates across many layers.

Let $H$ be a minibatch of activations of the layer
to normalize, arranged as a design matrix, with the activations for each example
appearing in a row of the matrix. To normalize $H$ , we replace it with: <br>
$H' = \cfrac{H - \mu}{\sigma}$
<br>where $\mu$ is  a vector containing the mean of each unit and $\sigma$ is a vector containing the standard deviation of each unit.

At training time, <br>
$\mu = \cfrac{1}{m} \sum\limits_{i}H_{i,:}$ <br>
and <br>
$\sigma = \sqrt{\delta + \cfrac{1}{m}\sum\limits_{i} (H - \mu)_i^2}$

 Crucially, **we back-propagate through
these operations** for computing the mean and the standard deviation, and for
applying them to normalize H . This means that the gradient will never propose
an operation that acts simply to increase the standard deviation or mean of
h i ; the normalization operations remove the effect of such an action and zero
out its component in the gradient. This was a major innovation of the batch
normalization approach.

At test time,$\mu$ and $\sigma$ may be replaced by running averages that were collected during training time. This allows the model to be evaluated on a single example, without needing to use definitions of $\mu$ and $\sigma$ that depend on an entire minibatch.

 Batch normalization acts to
standardize only the mean and variance of each unit in order to stabilize learning,
but allows the relationships between units and the nonlinear statistics of a single
unit to change.

Normalizing the mean and standard deviation of a unit can reduce the expressive
power of the neural network containing that unit. In order to maintain the
expressive power of the network, it is common to replace the batch of hidden unit
activations $H$ with $\gamma H' + \beta$ rather than simply the normalized $H'$ The variables $\gamma$ and $\beta$ are learned parameters that allow the new variable to have any mean and standard deviation.

 new parametrization can represent
the same family of functions of the input as the old parametrization, but the new
parametrization has different **learning dynamics.** <br>
 In the new parametrization, the mean of $\gamma H' + \beta$ is
determined solely by $\beta$ . The new parametrization is much easier to learn with
gradient descent.

#### 8.7.2 Coordinate Descent

If we minimize $f(x)$ with respect to a single variable $x_i$, then minimize it with respect to another variable $x_j$ and so on,
repeatedly cycling through all variables, we are guaranteed to arrive at a (local) minimum. This practice is known as **coordinate descent** , because we optimize one coordinate at a time. More generally, **block coordinate descent** refers to
minimizing with respect to a subset of the variables simultaneously

Coordinate descent makes the most sense when the different variables in the
optimization problem can be clearly separated into groups that play relatively
isolated roles, or when optimization with respect to one group of variables is
significantly more efficient than optimization with respect to all of the variables.

Coordinate descent is not a very good strategy when the value of one variable
strongly influences the optimal value of another variable, as in the function $f(x) = (x_1 - x_2)^2 + \alpha(x_1^2+x_2^2)$ where $\alpha$ is a positive constant.  The first term encourages
the two variables to have similar value, while the second term encourages them
to be near zero

#### 8.7.3 Polyak Averaging

Polyak averaging (Polyak and Juditsky 1992) consists of averaging together several
points in the trajectory through parameter space visited by an optimization
algorithm. 

if $t$ iterations of gradient descent visit points $\theta^{(1)},...,\theta^{(t)}$, then the output of the Polyak averaging algorithm is $\hat{\theta}^{(t)} = \frac{1}{t}\sum_i \theta^{(i)}$

The basic idea is that the
optimization algorithm may leap back and forth across a valley several times
without ever visiting a point near the bottom of the valley. The average of all of
the locations on either side should be close to the bottom of the valley though.

when applying Polyak averaging to **non-convex** problems, it is typical to use an exponentially decaying running average: <br>
$
    \hat{\theta}^{(t)} = \alpha \hat{\theta}^{(t-1)} + (1-\alpha)\theta^{(t)}
$

#### 8.7.4 Supervised Pretraining

#### 8.7.5 Designing Models to Aid Optimization

#### 8.7.6 Continuation Methods and Curriculum Learning